In [ ]:
!gmx --version

In [ ]:
%%bash

git clone https://github.com/m3g/packmol.git
cd packmol
git checkout v20.3.5
./configure
make

In [ ]:
!git clone https://github.com/zhmurov/trappeua
!mv trappeua/trappeua.ff .
!rm -rf trappeua

In [ ]:
!packmol/packmol < input/packmol.inp

In [ ]:
!cp input/topol.top .

In [ ]:
!gmx editconf -f conf.pdb -o conf.gro -box 4 4 4
!gmx grompp -f input/em.mdp -c conf.gro -o em.tpr
!gmx mdrun -deffnm em -nt 4

In [ ]:
!gmx grompp -f input/nvt.mdp -c em.gro -o nvt.tpr
!gmx mdrun -deffnm nvt -nt 4

In [ ]:
!gmx grompp -f input/npt.mdp -c nvt.gro -o npt.tpr
!gmx mdrun -deffnm npt -nt 4

In [ ]:
%%bash

ELECTRIC_FIELD=1.0
cp input/md_iso_E.mdp md_iso_${ELECTRIC_FIELD}.mdp
sed -i "s/ELECTRIC_FIELD/${ELECTRIC_FIELD}/g" md_iso_${ELECTRIC_FIELD}.mdp

In [ ]:
!gmx grompp -f md_iso_${ELECTRIC_FIELD}.mdp -c npt.gro -o md_iso_${ELECTRIC_FIELD}.tpr
!gmx mdrun -deffnm md_iso_${ELECTRIC_FIELD} -nt 4